In [12]:
from datetime import datetime
import backtrader as bt
import tushare as ts
import pandas as pd
from backtrader.feeds import PandasData

In [13]:
# tushare
def get_single_kdata(code, start='2020-01-01', end='2021-08-18', index=False):
    df = ts.get_k_data(code, autype='qfq', start=start, end=end, index=index)
    df['date'] = pd.to_datetime(df['date'])
    df['openinterest'] = 0
    return df[['date', 'open', 'high', 'low', 'close', 'volume', 'openinterest']]
# yahoo
# def get_single_kdata(code, start='2020-01-01', end='2021-08-18'):
#     df = bt.feeds.YahooFinanceData(
#             dataname='AAPL',
#             fromdate=datetime.strptime(start,'%Y-%m-%d'),
#             todate=datetime.strptime(end,'%Y-%m-%d'),
#             adjclose=True,
#             adjvolume=False)
#     return df

In [14]:
secu_lst = ['600000','000001']
kdata = {}
for secu in secu_lst:
    kdata[secu] = get_single_kdata(secu)
    
print(kdata['000001'].head())

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
        date   open   high    low  close     volume  openinterest
0 2020-01-02  16.65  16.95  16.55  16.87  1530231.0             0
1 2020-01-03  16.94  17.31  16.92  17.18  1116194.0             0
2 2020-01-06  17.01  17.34  16.91  17.07   862083.0             0
3 2020-01-07  17.13  17.28  16.95  17.15   728607.0             0
4 2020-01-08  17.00  17.05  16.63  16.66   847824.0             0


In [16]:
# 如果你想要的不同的start和end
# 自己定义
secu_lst = {'600000':{'start':'2020-01-01','end':'2020-07-18'},
            '000001':{'start':'2020-02-01','end':'2020-08-18'}}
kdata = {}
for secu in secu_lst.keys():
    print(secu)
    kdata[secu] = get_single_kdata(secu, secu_lst[secu]['start'], secu_lst[secu]['end']).reset_index(drop=True)
    
print(kdata['000001'].head())

600000
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
000001
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
        date   open   high    low  close     volume  openinterest
0 2020-02-03  13.99  14.70  13.99  13.99  2259194.0             0
1 2020-02-04  14.05  14.66  14.02  14.60  1706172.0             0
2 2020-02-05  14.59  14.89  14.32  14.63  1491380.0             0
3 2020-02-06  14.81  14.87  14.51  14.77  1185815.0             0
4 2020-02-07  14.60  14.69  14.41  14.62   924852.0             0


In [17]:
# 先拿大盘trading_dates
benchmark_start = min(secu_lst.values(), key=lambda x : x['start'])['start']
benchmark_end = max(secu_lst.values(), key=lambda x : x['end'])['end']
print(f"start: {benchmark_start}, end {benchmark_end}")

benchmark = '399905' # 中证500
kdata['benchmark'] = get_single_kdata(benchmark, benchmark_start, benchmark_end, index=True) 
print(kdata['benchmark'])
trading_dates = kdata['benchmark']['date'].tolist()
print(f"start: {trading_dates[0]}, end {trading_dates[-1]}")

start: 2020-01-01, end 2020-08-18
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
          date     open     high      low    close       volume  openinterest
0   2020-01-02  5306.67  5374.91  5288.17  5366.14  145894405.0             0
1   2020-01-03  5376.03  5395.26  5351.80  5380.64  128452021.0             0
2   2020-01-06  5365.22  5481.55  5355.02  5434.85  159587414.0             0
3   2020-01-07  5441.90  5499.84  5440.69  5499.84  144142722.0             0
4   2020-01-08  5481.98  5500.16  5409.57  5423.80  155406847.0             0
..         ...      ...      ...      ...      ...          ...           ...
147 2020-08-12  6580.01  6599.66  6395.01  6528.40  177889045.0             0
148 2020-08-13  6552.37  6592.12  6518.42  6552.66  162060515.0             0
149 2020-08-14  6542.60  6629.51  6506.03  6623.56  156875988.0             0
150 2020-08-17  6646.54  6752.32  6614.18  6744.55  201407357.0             0
151 2020-08-18  6747.61  6811.56  6737.53  6789.40  189

In [39]:
for secu in set(kdata.keys())-set(['benchmark']):
    print(secu)
    secu_kdata = kdata['benchmark'][['date']].merge(kdata[secu],how='left')
    secu_kdata['suspend'] = 0 
    secu_kdata.loc[secu_kdata['open'].isnull(), 'suspend'] = 1 # 标记为停盘日
    secu_kdata.set_index(['date'], inplace = True) # 设date为index
    end = secu_lst[secu]['end']
    secu_kdata.fillna(method='ffill',inplace = True) # start后的数据用前日数据进行补充
    secu_kdata.fillna(value = 0,inplace = True) #start前的数据用0补充
    secu_kdata.loc[(secu_kdata.index > end), 'suspend'] = 1
    print(secu_kdata)
    kdata[secu] = secu_kdata

600000
             open   high    low  close    volume  openinterest  suspend
date                                                                   
2020-01-02  12.47  12.64  12.45  12.47  516290.0           0.0        0
2020-01-03  12.57  12.63  12.47  12.60  380188.0           0.0        0
2020-01-06  12.52  12.65  12.42  12.46  410011.0           0.0        0
2020-01-07  12.51  12.60  12.46  12.50  284214.0           0.0        0
2020-01-08  12.41  12.45  12.25  12.32  352405.0           0.0        0
...           ...    ...    ...    ...       ...           ...      ...
2020-08-12  11.37  11.39  11.17  11.27  686395.0           0.0        1
2020-08-13  11.37  11.39  11.17  11.27  686395.0           0.0        1
2020-08-14  11.37  11.39  11.17  11.27  686395.0           0.0        1
2020-08-17  11.37  11.39  11.17  11.27  686395.0           0.0        1
2020-08-18  11.37  11.39  11.17  11.27  686395.0           0.0        1

[152 rows x 7 columns]
000001
             open   high  

In [40]:
class GetKdatas(object):
    def __init__(self, secu_lst, benchmark='000001'):
        """
        :parameter secu_lst: a dict contained stocks with starts and ends
        :parameter benchmark: the name of benchmark
        """
        self.secu_lst = secu_lst
        self.benchmark = benchmark

    @staticmethod
    def get_single_kdata(code, start='2020-01-01', end='2021-08-18', index=False):
        df = ts.get_k_data(code, autype='qfq', start=start, end=end, index=index)
        df['date'] = pd.to_datetime(df['date'])
        df['openinterest'] = 0
        return df[['date', 'open', 'high', 'low', 'close', 'volume', 'openinterest']]

    def get_all_kdata(self):
        kdata = {}
        for secu in set(self.secu_lst):
            secu_kdata = self.get_single_kdata(secu, self.secu_lst[secu]['start'], self.secu_lst[secu]['end'])
            kdata[secu] = secu_kdata.reset_index(drop=True)
        return kdata

    def merge_period(self):
        all_kdata = self.get_all_kdata()
        benchmark_start = min(self.secu_lst.values(), key=lambda x: x['start'])['start']
        benchmark_end = max(self.secu_lst.values(), key=lambda x: x['end'])['end']
        all_kdata['benchmark'] = self.get_single_kdata(self.benchmark, benchmark_start, benchmark_end,True)

        for secu in set(all_kdata.keys()) - set(['benchmark']):
            secu_kdata = all_kdata['benchmark'][['date']].merge(all_kdata[secu], how='left')
            secu_kdata['suspend'] = 0
            secu_kdata.loc[secu_kdata['open'].isnull(), 'suspend'] = 1  # 标记为停盘日
            secu_kdata.set_index(['date'], inplace=True)  # 设date为index
            end = secu_lst[secu]['end']
            secu_kdata.fillna(method='ffill', inplace=True)  # start后的数据用前日数据进行补充
            secu_kdata.fillna(value=0, inplace=True)  # start前的数据用0补充
            secu_kdata.loc[(secu_kdata.index > end), 'suspend'] = 1
            all_kdata[secu] = secu_kdata

        _ = all_kdata.pop('benchmark')
        return all_kdata

In [41]:
secu_lst = {'600000': {'start': '2020-01-01', 'end': '2020-07-18'},
            '000001': {'start': '2020-02-01', 'end': '2020-08-18'}}

kdata = GetKdatas(secu_lst).merge_period()
print(kdata)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
{'600000':              open   high    low  close    volume  openinterest  suspend
date                                                                   
2020-01-02  12.47  12.64  12.45  12.47  516290.0           0.0        0
2020-01-03  12.57  12.63  12.47  12.60  380188.0           0.0        0
2020-01-06  12.52  12.65  12.42  12.46  410011.0           0.0        0
2020-01-07  12.51  12.60  12.46  12.50  284214.0           0.0        0
2020-01-08  12.41  12.45  12.25  12.32  352405.0           0.0        0
...           ...    ...    ...    ...       ...           ...      ...
2020-08-12  11.37  11.39  11.17  11.27  686395.0           0.0        1
2020-08-13  11.37  11.39  11.17  11.27  686395.0           0.0        1
2020-08-14  11.37  11.39  11.17  11.27  686395.0           0.0        1
2020-08-17  11.37  11.39  11.17  11.27

In [42]:
cerebro = bt.Cerebro()
for secu in set(kdata.keys())-set(['benchmark']):
    df = PandasData(dataname=kdata[secu], fromdate=kdata[secu].index[0],todate=kdata[secu].index[-1])
    cerebro.adddata(df, name=secu) 

# 如果是本地csv，如果你的column是和默认一样的，那从datetime开始都不用写，否则必须指明
# data = btfeeds.GenericCSVData(
#     dataname='mydata.csv',
#     fromdate=benchmark_start,
#     todate=benchmark_end,
#     dtformat=('%Y-%m-%d'),
#     datetime=0,
#     high=1,
#     low=2,
#     open=3,
#     close=4,
#     volume=5,
#     openinterest=-1
# )

In [43]:
class PandasData_Extend(PandasData):
     # Add a 'suspend' line to the inherited ones from the base class
    lines = ('suspend',)
    # 现在是(open,0), ... , (openinterest,5)这6列，所以增加1
    # add the parameter to the parameters inherited from the base class
    params = (('suspend', 6),)
    
    # 如果是csv文档，openinterest 在 GenericCSVData index是7，所以1
    # params = (('suspend', 8),)

In [44]:
cerebro = bt.Cerebro()
# 设置初始资本为1 million
startcash = 10**6
cerebro.broker.setcash(startcash)
print(f"初始资金{cerebro.broker.getvalue()}")

初始资金1000000


In [45]:
class CommInfoPro(bt.CommInfoBase):

    params = (
        ('stamp_duty', 0.001),  # 印花税率
        ('stamp_duty_fe', 1),  # 最低印花税
        ('commission', 0.001),  # 佣金率
        ('commission_fee', 5),  # 最低佣金费
        ('stocklike', True), # 股票
        ('commtype', bt.CommInfoBase.COMM_PERC), # 按比例收
    )

    def _getcommission(self, size, price, pseudoexec):
        '''
        If size is greater than 0, this indicates a long / buying of shares.
        If size is less than 0, it idicates a short / selling of shares.
        '''

        if size > 0:  # 买入，不考虑印花税
            return max(size * price * self.p.commission, self.p.commission_fee)
        elif size < 0:  # 卖出，考虑印花税
            return max(size * price * (self.p.stamp_duty + self.p.commission), self.p.stamp_duty_fe)
        else:
            return 0  # just in case for some reason the size is 0.

        
cerebro.broker.addcommissioninfo(CommInfoPro())

In [46]:
# 原有
#self.buy(data, amount)
#self.sell(data, amount)

#改为
def downcast(amount, lot):
    return abs(amount//lot*lot)

print(downcast(418,100))
print(downcast(-418,100))
#self.buy(data, downcast(amount, lot))
#self.sell(data, downcast(amount, lot))

400
500


In [47]:
class multi_strategy(bt.Strategy):
    # 全局设定交易策略的参数
    params = (
        ('maperiod', 20),
    )

    def __init__(self):
        # 初始化交易指令
        self.order = None
        self.buy_lst = []

        # 添加移动均线指标，内置了talib模块
        # 循环计算每只股票的指标
        self.sma = {x: bt.ind.SMA(self.getdatabyname(x), period=self.p.maperiod) for x in self.getdatanames()}

    def prenext(self):
        pass
    
        
    def downcast(self, amount, lot):
        return abs(amount//lot*lot)
    
    
    def next(self):
        if self.order:  # 检查是否有指令等待执行,
            return

        # 回测最后一天不进行买卖
        if self.datas[0].datetime.date(0) == end_date:
            return 
        
        # 检查是否持仓
        self.log(f'{self.broker.getvalue():.2f}, {[(x, self.getpositionbyname(x).size) for x in self.buy_lst]}')
        if len(self.buy_lst) < 2:  # 没有持仓
            for secu in set(self.getdatanames()) - set(self.buy_lst):
                data = self.getdatabyname(secu)
                # 执行买入条件判断：收盘价格上涨突破20日均线
                # 不要在股票剔除日前一天进行买入
                if data.close > self.sma[secu] and \
                    data.datetime.date(1) < pd.Timestamp(secu_lst[secu]['end']):
                    # 执行买入
                    order_value = self.broker.getvalue()*0.48
                    order_amount = self.downcast(order_value/data.close[0], 100)
                    self.order = self.buy(data, size=order_amount, name=secu)
                    self.log(f"买{secu}, price:{data.close[0]:.2f}, amout: {order_amount}")
                    self.buy_lst.append(secu)
        elif self.position:
            now_lst = []
            for secu in self.buy_lst:
                data = self.getdatabyname(secu)
                # 执行卖出条件判断：收盘价格跌破20日均线，或者股票剔除
                if (data.close < self.sma[secu]) or \
                    (data.datetime.date(1) >= pd.Timestamp(secu_lst[secu]['end'])):
                    # 执行卖出
                    self.order = self.order_target_percent(data, 0, name=secu)
                    # 也可以用 self.sell(data, size = self.getposition(data).size)
                    # or self.sell(data, size = self.getpositionbyname(secu).size)
                    self.log(f"卖{secu}, price:{data.close[0]:.2f}, pct: 0")
                    continue
                now_lst.append(secu)
            self.buy_lst = now_lst.copy()

    def log(self, txt, dt=None):
        ''' 输出日志'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(f"""买入{order.info['name']}, 成交量{order.executed.size}，成交价{order.executed.price:.2f}""")
            elif order.issell():
                self.log(f"""卖出{order.info['name']}, 成交量{order.executed.size}，成交价{order.executed.price:.2f}""")
            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None

        
secu_lst = {'600000': {'start': '2020-01-01', 'end': '2021-07-20'},
            '000001': {'start': '2020-02-01', 'end': '2021-08-18'}}
# 拿对齐的数据
kdata = GetKdatas(secu_lst).merge_period()
kdata = dict(sorted(kdata.items()))

# 开始回测
cerebro = bt.Cerebro()
cerebro.addstrategy(multi_strategy)

for secu in kdata.keys():
    df = kdata[secu]
    data = PandasData_Extend(dataname=df, fromdate=df.index[0], todate=df.index[-1])
    cerebro.adddata(data, name=secu)
end_date = df.index[-1]

# 设置初始资本为1 million
startcash = 10**6
cerebro.broker.setcash(startcash)
print(f"初始资金{cerebro.broker.getvalue()}")
# 设置交易手续费
cerebro.broker.addcommissioninfo(CommInfoPro())

# 加入指标
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_sharpe')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_annrtn')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_dd')
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='_pyfolio')
# 运行回测系统
thestrats = cerebro.run()
# 获取回测结束后的总资金
portvalue = cerebro.broker.getvalue()
# 打印结果
print(f'结束资金: {round(portvalue, 2)}')

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
初始资金1000000
2020-02-06, 1000000.00, []
2020-02-06, 买000001, price:14.77, amout: 32400.0
2020-02-07, 买入000001, 成交量32400.0，成交价14.60
2020-02-07, 1000643.00, [('000001', 32400.0)]
2020-02-10, 996755.00, [('000001', 32400.0)]
2020-02-11, 1006151.00, [('000001', 32400.0)]
2020-02-12, 1005503.00, [('000001', 32400.0)]
2020-02-13, 1001615.00, [('000001', 32400.0)]
2020-02-14, 1013927.00, [('000001', 32400.0)]
2020-02-17, 1024943.00, [('000001', 32400.0)]
2020-02-18, 1019435.00, [('000001', 32400.0)]
2020-02-19, 1020731.00, [('000001', 32400.0)]
2020-02-20, 1032071.00, [('000001', 32400.0)]
2020-02-20, 买600000, price:11.23, amout: 44100.0
2020-02-21, 买入600000, 成交量44100.0，成交价11.23
2020-02-21, 1034829.00, [('000001', 32400.0), ('600000', 44100.0)]
2020-02-24, 1017315.00, [('000001', 32400.0), ('600000', 44100.0)]
2020-02-25, 1008072.00, [('

/tmp/ipykernel_1792850/157646190.py:29: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if self.datas[0].datetime.date(0) == end_date:
/tmp/ipykernel_1792850/157646190.py:40: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  data.datetime.date(1) < pd.Timestamp(secu_lst[secu]['end']):
/tmp/ipykernel_1792850/157646190.py:53: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  (data.datetime.date(1) >= pd.Ti

In [48]:
# cerebro.plot(style='candlestick')


ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/usr/local/lib/python3.8/dist-packages/matplotlib/dates.py)